# INICIO:
Este proyecto se centra en la detección de fraudes en transacciones con tarjetas de crédito utilizando un modelo de regresión logística. Dado el desafío de un conjunto de datos desbalanceado, aplicaremos técnicas de preprocesamiento para mejorar la precisión del modelo en la identificación de transacciones fraudulentas.


In [ ]:
# procesamiento
import pandas as pd
# escalan todos los datos
from sklearn.preprocessing import StandardScaler
#modelo
from sklearn.linear_model import LogisticRegression
# Separa los datos
from sklearn.model_selection import train_test_split
# metricas para evaluar el modelo
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
# visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('creditcard.csv')

In [ ]:
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [ ]:
# vemos la dimesionalidad
df.shape

(284807, 31)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
# La clase que nos interesa es 'Class'
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

Mierda podemos ver que los datos estan desvalaceados
hay mas 0 que 1.
como lo solucionaremos nose a ver que se puede hacer pa

In [ ]:
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# Realizamos predicion
y_pred = model.predict(X_test)

In [ ]:
#Verifico la matriz de Confusión
matriz_confusion = confusion_matrix(y_test, y_pred)
print(matriz_confusion)

[[85258    38]
 [   68    79]]


In [ ]:
# Predicion del modelo
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6752136752136753


### Vemos que el modelo es impreciso.
### Una posible causa es que los datos están desbalaceados.

In [ ]:
# Usamos la tecnica de balaceo solver="newton-g",class_weight="balaced"
model_2 = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg",class_weight="balanced")

In [ ]:
model_2.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


LogisticRegression(class_weight='balanced', random_state=1, solver='newton-cg')

In [ ]:
y2_pred = model_2.predict(X_test)

In [ ]:
def run_model_balanced(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg",class_weight="balanced")
    clf.fit(X_train, y_train)
    return clf

model_3 = run_model_balanced(X_train, X_test, y_train, y_test)
y3_pred = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


In [ ]:
precision_3 = precision_score(y_test, y3_pred)
print('Precisión del modelo:')
print(precision_3)
exactitud_3 = accuracy_score(y_test, y3_pred)
print('Exactitud del modelo:')
print(exactitud_3)

Precisión del modelo:
0.07170224411603722
Exactitud del modelo:
0.9799632503540372


In [ ]:
# cuarto modelo
df_copy = df.copy()
# para este vamos a hacer una muestra significativa de los datos de class = 0 osea no fraude

In [ ]:
Fraud_transaction = df_copy[df_copy['Class']==1]
Normal_transaction = df_copy[df_copy['Class']==0]

In [ ]:
Normal_transaction = Normal_transaction.sample(492)
print(Normal_transaction.shape)

(492, 31)


In [ ]:
muestra = pd.concat((Normal_transaction, Fraud_transaction), axis=0)

In [ ]:
muestra.shape

(984, 31)

In [ ]:
X = muestra.drop('Class', axis=1)
y = muestra['Class']

In [ ]:
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, test_size=0.1, random_state=1, stratify=y)

In [ ]:
model_4 = LogisticRegression()
model_4.fit(X_train_4, y_train_4)


LogisticRegression()

In [ ]:
y4_pred = model_4.fit(X_train_4, y_train_4).predict(X_test_4)

In [ ]:
precision_4 = precision_score(y_test, y4_pred)
print('Precisión del modelo:')
print(precision_4)
exactitud_4 = accuracy_score(y_test, y4_pred)
print('Exactitud del modelo:')
print(exactitud_4)

Precisión del modelo:
0.9361702127659575
Exactitud del modelo:
0.9090909090909091


# Conclusión:
En resumen, la regresión logística, combinada con técnicas de manejo de datos desbalanceados, ha mostrado ser eficaz en la detección de fraudes.